Bibliotecas

In [2]:
from xmlrpc import client
import dash
from dash import html, dcc, html, dash_table, Output, Input, State
from dash.dependencies import Input, Output, State
import pandas as pd
import plotly.express as px
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build   
import gspread

Google API integration

In [27]:


SCOPES = ['https://www.googleapis.com/auth/spreadsheets',
          'https://www.googleapis.com/auth/drive']
SHEET_ID = 'dasd' # Replace with desired Google Sheet ID
SHEET_NAME = 'Acompanhamento Setores'

def get_data_from_sheets():
    try:
        # Load credentials
        creds = Credentials.from_service_account_file('Credentials.json', scopes=SCOPES)
        client = gspread.authorize(creds)

        # Open sheet and worksheet
        worksheet = client.open_by_key(SHEET_ID).worksheet(SHEET_NAME)

        # Fetch all values
        data = worksheet.get_all_values()

        # Convert to DataFrame (first row = headers)
        df = pd.DataFrame(data[1:], columns=data[0])
        return df

    except Exception as e:
        print(f"Error fetching data from Google Sheets: {e}")
        return pd.DataFrame()


App - start/interface

In [35]:
app = dash.Dash(__name__, suppress_callback_exceptions=True)
app.layout = html.Div([
    html.H1("Dashboard App"),
    
    # Navigation Menu
    dcc.Dropdown(
        id="menu-dropdown",
        options=[
            {"label": "Dashboard", "value": "dashboard"},
            {"label": "Alterations", "value": "alterations"},
            {"label": "Google Sheets Data", "value": "sheet"}
        ],
        value="dashboard",  # home page
        clearable=False
    ),
    
    html.Div(id="page-content", style={"margin-top": "20px"})
])

# Page rendering 
@app.callback(Output("page-content", "children"), Input("menu-dropdown", "value"))
def render_page(selected_page):
    if selected_page == "dashboard":
        return html.Div([
            html.H2("Dashboard Page"),
            dcc.Dropdown(
                id="region-dropdown",
                options=[
                    {"label": "Region 1", "value": "Region 1"},
                    {"label": "Region 2", "value": "Region 2"},
                    {"label": "Region 3", "value": "Region 3"}
                ],
                value="Region 1"
            ),
            dcc.Graph(id="dashboard-graph"),
            html.Div(id="output-container")
        ])
    elif selected_page == "alterations":
        return html.Div([
            html.H2("Alterations Page"),
            html.P("This is the alterations page content.")
        ])
    elif selected_page == "sheet":
        df = get_data_from_sheets()
        if df.empty:
            return html.Div("No data found in Google Sheets.")
        return html.Div([
            html.H2("Google Sheets Data"),
            dcc.Graph(
                figure=px.bar(df, x="Category", y="Value", title="Data from Google Sheets")
            )
        ])
    else:
        return html.Div([html.H3("404 - Page not found")])

# Dashboard update
@app.callback(
    [Output("dashboard-graph", "figure"),
     Output("output-container", "children")],
    Input("region-dropdown", "value")
)
def update_dashboard(selected_region):
    df = get_data_from_sheets()
    if df.empty:
        return {}, "No data available."
    filtered_df = df[df["Region"] == selected_region]
    fig = px.bar(filtered_df, x="Category", y="Value", title=f"Data for {selected_region}")
    output_text = f"Displaying {len(filtered_df)} records for {selected_region}"
    return fig, output_text

if __name__ == "__main__":
    app.run(debug=True)


Error fetching data from Google Sheets: <Response [404]>
